In [ ]:
#For the LaTeX equations (such as eqnarray) in this document to work, include the following in file
#~/.jupyter/_config.yml
#
#parse:
#  myst_enable_extensions:  # default extensions to enable in the myst parser. See https://myst-parser.readthedocs.io/en/latest/using/syntax-optional.html
#     - amsmath
#
#(the default ~/.jupyter/_config.yml will have amsmath commented out)
#
#This notebook uses sympy and sparse linear algegra

# Deriving Third-order Lax-Wendroff
Start from the Taylor series in time and convert temporal to spatial derivatives:
\begin{eqnarray}
\psi_{j}^{n+1}	&=&	\psi_{j}^{n} + \Delta t\frac{\partial\psi_{j}^{n}}{\partial t}
                 + \chi_2\frac{\Delta t^{2}}{2!}\frac{\partial^{2}\psi_{j}^{n}}{\partial t^{2}}
                 + \chi_3\frac{\Delta t^{3}}{3!}\frac{\partial^{3}\psi_{j}^{n}}{\partial t^{3}} + \left(\Delta t^{4}\right) \\
%
	            &=&	\psi_{j}^{n} - u\Delta t\frac{\partial\psi_{j}^{n}}{\partial x}
                + \chi_2 u^{2}\frac{\Delta t^{2}}{2!}\frac{\partial^{2}\psi_{j}^{n}}{\partial x^{2}}
                - \chi_3 u^{3}\frac{\Delta t^{3}}{3!}\frac{\partial^{3}\psi_{j}^{n}}{\partial x^{3}} + O\left(\Delta t^{4}\right)
\end{eqnarray}
The limiters, $\chi_2$ and $\chi_3$ limit the second- and third-order corrections respectively.

In order to find the derivatives, consider the cubic polinomial
$$
\psi=ax^{3}+bx^{2}+cx+d
$$
so that
$$
\psi^{\prime} = 3ax^{2} + 2bx + c, \hspace{1cm} \psi^{\prime\prime} = 6ax+2b, \hspace{1cm} \psi^{\prime\prime\prime} = 6a.
$$
Then we assume that position $i$ is at $x=0$, then we get:
$$
\psi_{i} = d, \hspace{1cm} \psi_{i}^{\prime} = c, \hspace{1cm}
\psi_{i}^{\prime\prime} = 2b, \hspace{1cm} \psi_{i}^{\prime\prime\prime} = 6a.
$$
Substituting positions $x_{i-2}=-2\Delta x$, $x_{i-1}=-\Delta x$ and $x_{i+1}=\Delta x$ into the cubic polynomial gives
\begin{eqnarray}
\psi_{i-2}	&=&	-8a\Delta x^{3}+4b\Delta x^{2}-2c\Delta x+d \\
\psi_{i-1}	&=&	-a\Delta x^{3}+b\Delta x^{2}-c\Delta x + d \\
\psi_{i}	&=&	d \\
\psi_{i+1}	&=&	a\Delta x^{3}+b\Delta x^{2}+c\Delta x+ d \\
\end{eqnarray}
which can be solving using Gaussian elimination to find $a$, $b$ and $c$. This can assume $\Delta x = 1$ without loss of generality:

In [ ]:
import sympy as sy
from sympy.matrices import Matrix, MatrixSymbol
from fractions import Fraction as Fr
polyCoeffs = sy.Matrix(sy.symarray("polyCoeffs", (4,))) # Coefficients a,b,c,d of the polynomial
psi = sy.Matrix(sy.symarray("psi", (4,)))               # Grid point values at i-2, i-1, i and i+1
M = Matrix([[-8,4,-2,1], [-1,1,-1,1], [0,0,0,1], [1,1,1,1]])
polyCoeffs = M.solve(psi)
ddx = polyCoeffs[2]
d2dx2 = 2*polyCoeffs[1]
d3dx3 = 6*polyCoeffs[0]
print('d/dx =', ddx, '\nd2/dx2 =', d2dx2, '\nd3/dx3 =', d3dx3)

In [ ]:
# From this we can work out the explicit LW3e  increment
[c, chi2, chi3] = sy.symbols("c, chi2, chi3")   # The Courant number
LW3e = - c*ddx + chi2*c**2/2*d2dx2 - chi3*c**3/6*d3dx3
print('LW3e increment is', LW3e)
print('LW3e increment as coefficients of grid points is\n', sy.collect(sy.expand(LW3e), psi))

The implicit LW3i scheme is
\begin{eqnarray}
\psi_{j}^{n} &=& \psi_{j}^{n+1} - \Delta t\frac{\partial\psi_{j}^{n+1}}{\partial t}
        +\frac{\Delta t^{2}}{2!}\frac{\partial^{2}\psi_{j}^{n+1}}{\partial t^{2}}
        -\frac{\Delta t^{3}}{3!}\frac{\partial^{3}\psi_{j}^{n+1}}{\partial t^{3}}
        +\left(\Delta t^{4}\right) \\
\implies \psi_j^{n+1} &=& \psi_j^{n} + \Delta t\frac{\partial\psi_{j}^{n+1}}{\partial t}
        -\frac{\Delta t^{2}}{2!}\frac{\partial^{2}\psi_{j}^{n+1}}{\partial t^{2}}
        +\frac{\Delta t^{3}}{3!}\frac{\partial^{3}\psi_{j}^{n+1}}{\partial t^{3}} \\
        &=& \psi_j^{n} -u \Delta t\frac{\partial\psi_{j}^{n+1}}{\partial x}
        -u\frac{\Delta t^{2}}{2!}\frac{\partial^{2}\psi_{j}^{n+1}}{\partial x^{2}}
        -u\frac{\Delta t^{3}}{3!}\frac{\partial^{3}\psi_{j}^{n+1}}{\partial x^{3}} \\
\end{eqnarray}

In [ ]:
# The implicit LW3i scheme
LW3i = - c*ddx - c**2/2*d2dx2 - c**3/6*d3dx3
print('LW3i increment is ', LW3i)
print('LW3i increment as coefficients of grid points is\n', sy.collect(sy.expand(LW3i), psi))

In [ ]:
# Stability Analysis of various schemes
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
from numpy import exp

def psi0(kdx):
    return exp(-2*1j*kdx)

def psi1(kdx):
    return exp(-1j*kdx)

def psi2(kdx):
    return 1

def psi3(kdx):
    return exp(1j*kdx)

def A_LW3(c, a, kdx, chi2 = 1, chi3 = 1):
    """Amplification factor for LW3 for Courant number c, off-center a, wave kdx
       chi2 is the limiter on the 2nd order temporal correction and chi3 on the 3rd order"""
    cSqr = chi2*c**2
    cCub = chi3*c**3
    return (1 + (1-a)*(psi0(kdx)*(cCub/6 - c/6) + psi1(kdx)*(-cCub/2 + cSqr/2 + c)\
                     + psi2(kdx)*(cCub/2 - cSqr - c/2) + psi3(kdx)*(-cCub/6 + cSqr/2 - c/3)))/\
            (1-a*(psi0(kdx)*(cCub/6 - c/6) + psi1(kdx)*(-cCub/2 - cSqr/2 + c)
                + psi2(kdx)*(cCub/2 + cSqr - c/2) + psi3(kdx)*(-cCub/6 - cSqr/2 - c/3)))

a =     [0, 0.5, 1]
chis =  [{"chi2":1, "chi3":1}, {"chi2":1, "chi3":0}]
titles = [["3rd order Explicit", "3rd order ImEx05", "3rd order Implicit"], 
          ["2nd order in time", "2nd order in time", "2nd order in time"]]

kdxs = np.linspace(1e-6, 2*np.pi, 37)
cs = np.arange(0, 5.1, 0.1) #10**(np.linspace(-1, 1, 81))
magA = np.zeros([len(kdxs), len(cs)])
for ich in range(len(chis)):
    fig,axs = plt.subplots(1,len(a), figsize=(12,4), layout='constrained')
    if ich == 0:
        fig.suptitle("Lax-Wendroff Amplification Factor Magnitudes")
    for i in range(len(a)):
        for ic in range(len(cs)):
            c = cs[ic]
            for ik in range(len(kdxs)):
                kdx = kdxs[ik]
                magA[ik,ic] = abs(A_LW3(c, a[i],kdx, **(chis[ich])))
        axplot = axs[i].contourf(cs, kdxs,magA, np.arange(0, 2.1, 0.1))
        axs[i].axvline(x=1, color="black", linestyle=":")
        axs[i].axvline(x=2, color="black", linestyle=":")
        fig.colorbar(axplot,ax=axs[i], orientation='horizontal')
        axs[i].contour(cs, kdxs, magA, [0, 1], colors=['k', 'k'])
        axs[i].set(xlabel=r'$c$', ylabel=r'$k\Delta x$', title = titles[ich][i])

    plt.show()

## Implemented of an Implicit Scheme using a Sparse Matrix
The scheme is the second-order in time, third-order in space Lax-Wendroff implicit scheme:
$$
\psi_j^{n+1} = \psi_j^{n} - 
    \frac{c}{6}\left(\psi_{j-2}^{n+1} - 6 \psi_{j-1}^{n+1} + 3 \psi_j^{n+1} + 2 \psi_{j+1}^{n+1}\right)
   - \frac{c^2}{2} \left(\psi_{j-1}^{n+1} - 2 \psi_j^{n+1} + \psi_{j+1}^{n+1}\right)
$$
which is stable for Courant numbers away from one. This can be written as:
$$
   \frac{c}{6}\psi_{j-2}^{n+1}
- \frac{c}{2}\left(2- c\right)\psi_{j-1}^{n+1}
+ \left(1 + \frac{c}{2} - c^2 \right)\psi_j^{n+1}
+ \frac{c}{6}\left(2 + 3c \right)\psi_{j+1}^{n+1}
= \psi_j^{n}
$$
and then as a periodic matrix equation:
$$
\left( \begin{array}{ccccccccc}
1+\frac{c}{2}-c^2 & \frac{c}{3}+\frac{c^2}{2}& 0                        & \cdots                   &                           &            & 0                & \frac{c}{6}      & -c+\frac{c^2}{2} \\
-c+\frac{c^2}{2}  & 1+\frac{c}{2}-c^2        & \frac{c}{3}+\frac{c^2}{2}& 0                        & \cdots                    &            &                  &  0               & \frac{c}{6} \\
\frac{c}{6}       & -c+\frac{c^2}{2}         & 1+\frac{c}{2}-c^2        & \frac{c}{3}+\frac{c^2}{2}& 0                         &\cdots      &                  &                  & 0 \\
0                 & \frac{c}{6}              & -c+\frac{c^2}{2}         & 1+\frac{c}{2}-c^2        & \frac{c}{3}+\frac{c^2}{2} & 0          & \cdots           &                  & 0  \\
\vdots            & \vdots                   & \vdots                   &  \vdots                  & \vdots                    & \vdots & \vdots & \vdots & \vdots \\
0                 &                          &                          &                          & 0                         & \frac{c}{6} & -c+\frac{c^2}{2} & 1+\frac{c}{2}-c^2&\frac{c}{3}+\frac{c^2}{2}\\
\frac{c}{3}+\frac{c^2}{2}& 0                 &                          &                          &                          & 0                         & \frac{c}{6} & -c+\frac{c^2}{2} & 1+\frac{c}{2}-c^2\\
\end{array}\right)
\left( \begin{array}{c}
\phi_0^{n+1} \\ \phi_1^{n+1} \\ \phi_2^{n+1} \\ \phi_3^{n+1} \\ \vdots\\ \phi_{n_x-2}^{n+1} \\ \phi_{n_x-1}^{n+1} \\
\end{array}\right)
=
\left( \begin{array}{c}
\phi_0^{n} \\ \phi_1^{n} \\ \phi_2^{n} \\ \phi_3^{n} \\ \vdots\\ \phi_{n_x-2}^{n} \\ \phi_{n_x-1}^{n} \\
\end{array}\right)
$$

In [ ]:
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
import numpy as np

def LW32i(phi, c):
    """Lax Wendroff advection of profile phi with Courant number c for one time step
       third-order is space, second-order in time, implicit. Periodic boundary conditions"""
    nx = len(phi)
    # The sparse matrix for the implicit solve, defining the diagonals
    M = diags([c/3+c**2/2, # The bottom left corner
               c/6*np.ones(nx-2), # The diagonal for j-2
               -0.5*c*(2-c)*np.ones(nx-1), # The diagonal for j-1
               (1 + 0.5*c - c**2)*np.ones(nx), # The diagonal
               c/6*(2 + 3*c)*np.ones(nx-1), # The diagonal for j+1
               c/6*np.ones(2), # the diagonal next to the top right  corner
               -c+0.5*c**2], # the top right corner
               [-nx+1, -2, -1, 0, 1, nx-2, nx-1], # the locations of each of the diagonals
               shape=(nx,nx), format = 'csr')
    #print('M = ', M.toarray())
    return spsolve(M, phi)

# Parameters for one revolution of the periodic domain
nt = 12
dt = 1/nt
nx = 20
dx = 1/nx
c = nx/nt
plotFreq = 3
print('c =', c, 'dx =',dx, 'dt =', dt, 'nt =', nt, 'nx =', nx)

x = np.arange(0, 1, dx)
phi = np.where(x<0.5, 1., 0.)
plt.plot(x, phi, 'k', label = 't=0')
for it in range(nt):
    phi = LW32i(phi, c)
    if (it+1)%plotFreq == 0:
        plt.plot(x, phi, label = 't='+str(round((it+1)*dt, 2)))
plt.legend()
plt.show()